In [17]:
import pandas as pd
import re

In [18]:
main = pd.read_excel('통합의약품.xlsx')
main

,제품명,제형,용량,대분류,BIT,ATC코드,성분명,제품코드,주성분코드,구매처,ICD,복용법,금기대상,약품 보관법,일상 상호작용,의약품 상호작용,이상 반응,약품이미지(URL)
0,판콜에이내복액,액제,NaN,호흡기계,종합감기약,R05X,"chlorpheniramine, pseudoephedrine",642703531,D92500ALQ,상비,"콧물, 코막힘, 재채기, 기침, 인후통, 가래, 오한, 발열, 두통, 관절통, 근육...",성인 1회 30ml 1일 3회 식후 30분에 복용,"이 약 및 이 약의 구성성분에 대한 과민반응 및 그 병력이 있는 사람 , 이 약 및...",NaN,"충분한 수분섭취를 유지하세요., 음주를 삼가하세요.",NaN,NaN,https://encrypted-tbn0.gstatic.com/images?q=tb...
1,판피린티정,정제형,NaN,호흡기계,종합감기약,R05X,OTHER COLD PREPARATIONS,626500060,A04400ATB,상비,"콧물, 코막힘, 재채기, 인후통, 기침, 가래, 오한, 발열, 관절통, 두통, 근육...","성인 1회 1정, 1일 3회 식후 30분에 복용","만3개월 미만의 영아, 다른 해열진통제, 감기약 복용 시 천식을 일으킨 적이 있는 ...",NaN,"음주를 삼가하세요., 하루 3g 이상의 비타민 C 섭취를 권장하지 않아요.",NaN,NaN,https://www.druginfo.co.kr/drugimg/12192.jpg
2,신신파스아렉스 대형,플라스타,NaN,근골격계,외용 소염진통제,M02AC,Preparations with salicylic acid derivatives,643800460,B26300CPL,상비,"어깨걸림, 허리통, 신경통, 류마티스, 타박상, 염좌, 근육통, 관절통 치료",1일 1~1회 환부(질환 부위)에 부착,30개월 이하의 유아,NaN,NaN,NaN,NaN,https://www.druginfo.co.kr/drugimg/package/P20...
3,신신파스아렉스 중형,플라스타,NaN,근골격계,외용 소염진통제,M02AC,Preparations with salicylic acid derivatives,643800450,B26300CPL,상비,"어깨걸림, 허리통, 신경통, 류마티스, 타박상, 염좌, 근육통, 관절통 치료",1일 1~1회 환부(질환 부위)에 부착,30개월 이하의 유아,NaN,NaN,NaN,NaN,https://www.druginfo.co.kr/drugimg/package/P20...
4,제일쿨파프,카타플라스마,NaN,근골격계,외용 소염진통제,M02AC,Preparations with salicylic acid derivatives,645401460,341700CPO,상비,"어깨걸림, 허리통, 신경통, 류마티스, 타박상, 염좌, 근육통, 관절통 치료",1일 1~2회 환부에 부착,"30개월 이하의 유아, 상처부위, 피부염, 습진 부위",NaN,NaN,NaN,NaN,https://www.druginfo.co.kr/drugimg/package/P16...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,야스민정,정제형,NaN,비뇨생식기관 및 성호르몬,경구피임제,G03AA12,Drospirenone and ethinylestradiol,641100710,557000ATB,전문,피임,"1일 1정씩 21일간 표시된 순서에 따라 복용하고, 7일간 휴약","에스트로겐 의존성 종양 환자, 뇌혈관 또는 관상동맥질환 환자, 혈전증의 전구증상이 ...",NaN,엽산 보충제 복용이 도움을 줄 수 있어요.,NaN,NaN,https://www.druginfo.co.kr/drugimg/49918.jpg
177,트라젠타정,정제형,5mg,소화기관 및 대사,DPP-4 억제제,A10BH05,linagliptin,653501380,616401ATB,전문,제2형 당뇨병 환자의 혈당 조절 향상,1일 1회 5mg 식사에 관계없이 투여,제1형 당뇨병 또는 당뇨성 케톤산증 환자,NaN,NaN,아이오브릭스주300,기능적 신부전에 의해 유산 산성증 촉진,https://www.druginfo.co.kr/drugimg/203492.jpg
178,엑스포지정,정제형,5/160mg,심혈관계,칼슘채널길항제 / 안지오텐신 II 수용체 길항제,C09DB01,Valsartan and amlodipine,653600850,492800ATB,전문,암로디핀 또는 발사르탄 단독요법으로 혈압이 적절하게 조절되지 않는 본태성 고혈압,"1일 1회 1정으로, 식사와 관계없이 물과 함께 복용, 가능하면 매일 같은 시간에 ...","이 약의 주성분 및 디히드로피리딘계 유도체에 과민증이 있는 환자, 임부 또는 임신하...",NaN,NaN,NaN,NaN,https://www.druginfo.co.kr/drugimg/50273_1.jpg
179,탈시드정,정제형,NaN,소화기관 및 대사,제산제 & 흡착제,A02AX,"Antacids, other combinations",652100460,171402ATB,전문,"위.십이지장궤양, 위산과다, 급.만성 위염 치료",1회 0.5-1.0g 1일 4회 식간 및 취침시에 씹어서 경구 투여,"인산염 결핍 환자, 심기능 장애 환자, 설사 환자, 고마그네슘혈증 환자, 신장애 환자",NaN,NaN,NaN,NaN,https://www.druginfo.co.kr/drugimg/31750.jpg


In [26]:
def str_particle(text):
    if text.endswith('한다'):
        base = text[:-2].strip()
        noun = base.split()[-1]
        last_char = noun[-1]
        code = ord(last_char) - 0xAC00
        jong = code % 28
        particle = '을' if jong > 0 else '를'
        return f"{base} {particle} 한다"
    return text

# 요약 처리 함수
def simplify_icd(icd_text):
    text = str(icd_text).strip()

    # 규칙 1 : 쉼표로 나뉘는 항목이 5개 이상이면 괄호 처리
    parts = [p.strip() for p in text.split(',')]
    if len(parts) >= 5:
        symptoms = ', '.join(parts[:5])
        return f"{parts[0]} 등 다양한 증상({symptoms})에 사용한다."

    # 규칙 2 : "~한다" 로 끝나면 조사 붙이기
    if text.endswith('한다'):
        return str_particle(text)

    return text

In [27]:
# ICD 컬럼 적용
main['ICD_요약'] = main['ICD'].apply(simplify_icd)

# 출력
print(main[['ICD', 'ICD_요약']].head(10))

                                                 ICD  \
0  콧물, 코막힘, 재채기, 기침, 인후통, 가래, 오한, 발열, 두통, 관절통, 근육...   
1  콧물, 코막힘, 재채기, 인후통, 기침, 가래, 오한, 발열, 관절통, 두통, 근육...   
2         어깨걸림, 허리통, 신경통, 류마티스, 타박상, 염좌, 근육통, 관절통 치료   
3         어깨걸림, 허리통, 신경통, 류마티스, 타박상, 염좌, 근육통, 관절통 치료   
4         어깨걸림, 허리통, 신경통, 류마티스, 타박상, 염좌, 근육통, 관절통 치료   
5                           소화불량, 식욕감퇴, 과식, 체함, 소화촉진   
6                           소화불량, 식욕감퇴, 과식, 체함, 소화촉진   
7                           소화불량, 식욕감퇴, 과식, 체함, 소화촉진   
8                           소화불량, 식욕감퇴, 과식, 체함, 소화촉진   
9                  발열, 두통, 신경통, 근육통, 월경통, 염좌통, 치통 치료   

                                            ICD_요약  
0        콧물 등 다양한 증상(콧물, 코막힘, 재채기, 기침, 인후통)에 사용한다.  
1        콧물 등 다양한 증상(콧물, 코막힘, 재채기, 인후통, 기침)에 사용한다.  
2  어깨걸림 등 다양한 증상(어깨걸림, 허리통, 신경통, 류마티스, 타박상)에 사용한다.  
3  어깨걸림 등 다양한 증상(어깨걸림, 허리통, 신경통, 류마티스, 타박상)에 사용한다.  
4  어깨걸림 등 다양한 증상(어깨걸림, 허리통, 신경통, 류마티스, 타박상)에 사용한다.  
5   소화불량 등 다양한 증상(소화불량, 식욕감퇴, 과식, 체함, 소화촉진)에 사용한다.  
6   소화불량 등 다

In [28]:
main.to_excel('ICD요약.xlsx', index=False)

PermissionError: [Errno 13] Permission denied: 'ICD요약.xlsx'